# Synchronized tdmclient interactive session
## Advanced features

This notebook illustrates advanced use of `tdmclient.notebook`, such as explicit variable fetching or sending, the selection of a robot when multiple robots are available, disconnection and reconnection, and direct access to the objects of `tdmclient`.

As usual, make sure the latest version of tdmclient is installed (you can skip this step if you did it recently):

In [ ]:
%pip install --upgrade --quiet tdmclient

### Connection and disconnection

Import the required class.

In [ ]:
import tdmclient.notebook

Usually at this stage, we would connect to the first robot, assuming there is just one. It's also possible to display the list of robots:

In [ ]:
await tdmclient.notebook.list()

When you start the notebook session, you can add options to `tdmclient.notebook.start` to specify which robot to use. Robots can be identified by their id, which is unique hence unambiguous but difficult to type and remember, or by their name which you can define yourself.

Since we don't know the id or name of your robot, we'll cheat by picking the actual id and name of the first robot. To get the list of robots (or nodes), instead of `tdmclient.notebook.list` as above where the result is displayed in a nice list of properties, we call `tdmclient.notebook.get_nodes` which returns a list.

In [ ]:
nodes = await tdmclient.notebook.get_nodes()
id_first_node = nodes[0].id_str
name_first_node = nodes[0].props["name"]
print(f"id: {id_first_node}")
print(f"name: {name_first_node}")

Then you can specify the robot id:

In [ ]:
await tdmclient.notebook.start(node_id=id_first_node)

We want to show you how to use the robot's name instead of its id, but first we must close the connection:

In [ ]:
await tdmclient.notebook.stop()

Now the robot is available again.

In [ ]:
await tdmclient.notebook.start(node_name=name_first_node)

### Variable exchange

The synchronization of Python variables which have the same name as the robot's variables is performed automatically only when they're accessed directly in cells. This isn't automatically the case for functions, to let you decide when it's efficient to receive or send values to the robot. There are two ways to do it:
- Function `get_var("var1", "var2", ...)` retrieves the value of variables `var1`, `var2` etc. and returns them in a tuple in the same order. The typical use is to unpack the tuple directly into variables in an assignment: `var1,var2,...=get_var("var1","var2",...)`; beware to have a trailing comma if you retrieve only one variable, else you'll get a plain assignment of the tuple itself.<br>Function `set_var(var1=value1,var2=value2,...)` sends new values to the robot.<br>Here is an example which sets the color of the robot to red or blue depending on its temperature:

In [ ]:
def f(temp_limit=30):
    temperature, = get_var("temperature")
    if temperature > temp_limit * 10:
        set_var(leds_top=[32, 0, 0])
    else:
        set_var(leds_top=[0, 10, 32])

f(28)

- To synchronized global variables whose names match the robot's, the function can be decorated with `@tdmclient.notebook.sync_var`. The effect of the decorator is to extend the function so that these variables are fetched at the beginning and sent back to the robot before the function returns. Here is the same example:

In [ ]:
@tdmclient.notebook.sync_var
def f(temp_limit=30):
    global temperature, leds_top
    if temperature > temp_limit * 10:
        leds_top = [32, 0, 0]
    else:
        leds_top = [0, 10, 32]

f(28)

### Direct use of node objects

Once connected, the node object used to communicate with the robot can be obtained with `get_node()`:

In [ ]:
robot = tdmclient.notebook.get_node()

Then all the methods and properties defined for `ClientAsyncCacheNode` objects can be used. For example, you can get the list of its variables:

In [ ]:
robot_variables = list(await robot.var_description())
robot_variables

Or set the content of the scratchpad, used by the tdm to share the source code amoung all the clients. No need to use the actual source code, you can set it to any string. Check then in Aseba Studio.

In [ ]:
await robot.set_scratchpad("Hello from a notebook, Studio!")

The client object is also available as a `ClientAsync` object:

In [ ]:
client = tdmclient.notebook.get_client()

The client object doesn't have many intersting usages, because there are simpler alternatives with higher-level functions. Let's check whether the tdm is local:

In [ ]:
client.localhost_peer